# Assignment notebook: Logistic regresion

import modules

In [3]:
# System tools
import os
import sys
import argparse
sys.path.append(os.getcwd())

# Data tools
import numpy as np
from tqdm import tqdm
import pandas as pd
from random import sample
from itertools import chain

# Cifar-10 data
from tensorflow.keras.datasets import cifar10

# Image manipulation tools
import cv2

# Import sklearn metrics
from sklearn import metrics
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer

2022-05-18 22:11:09.216923: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-18 22:11:09.216955: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


__Loading data__

In [4]:
# > Load file_list to be certain that X data will be in the same order as y 
# Get the filepath
filepath = os.path.join("..", "in", "processed_data", "file_list.csv")
# Reading the filepath 
file_list = pd.read_csv(filepath)

In [5]:
# Define list of filenames
y_filenames = file_list["files"][:10]
X = []
for file in tqdm(y_filenames):
    filepath = os.path.join("..", "in", "np_arrays", file)
    loaded_array = np.load(filepath)
    X.append(loaded_array)

# Convert neural network data to array 
X_array = np.array(X)

100%|██████████| 10/10 [00:00<00:00, 22.19it/s]


In [6]:
X_array.shape

(10, 210, 210, 3)

In [60]:
def load_data(nr_files, random = True):
    # > load y data 
    filepath = os.path.join("..", "in", "processed_data", "y_data.npy")
    # Load array
    y = np.load(filepath)
    
    # > Load file_list to be certain that X data will be in the same order as y 
    # Get the filepath
    filepath = os.path.join("..", "in", "processed_data", "file_list.csv")
    # Reading the filepath 
    file_list = pd.read_csv(filepath)
    
    # Choose which files to load
    if random == True: 
        y_filenames = sample(file_list["files"].tolist(), nr_files)
    else:
        y_filenames = file_list["files"][:nr_files]
    
    # Define empthy list 
    X = []
    # Iterate over images to load as arrays
    for file in tqdm(y_filenames):
        # Get filepath for image
        filepath = os.path.join("..", "in", "np_arrays", file)
        # Load array
        loaded_array = np.load(filepath)
        # Append to list
        X.append(loaded_array)

    # Convert neural network data to array 
    X_array = np.array(X)
    
    return X_array, y

In [79]:
def load_data(nr_files):
    # > load y data 
    filepath = os.path.join("..", "in", "processed_data", "y_data.npy")
    # Load array
    y = np.load(filepath)
    # Choose relevant y data  
    y = y[:nr_files]
    
    # > Load file_list to be certain that X data will be in the same order as y 
    # Get the filepath
    filepath = os.path.join("..", "in", "processed_data", "file_list.csv")
    # Reading the filepath 
    file_list = pd.read_csv(filepath)
    
    # Choose which files to load
    y_filenames = file_list["files"][:nr_files]
    
    # Define empthy list 
    X = []
    # Iterate over images to load as arrays
    for file in tqdm(y_filenames):
        # Get filepath for image
        filepath = os.path.join("..", "in", "np_arrays", file)
        # Load array
        loaded_array = np.load(filepath)
        # Append to list
        X.append(loaded_array)

    # Making sure that both X and y are numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Splitting data 
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

In [80]:
X_train, X_test, y_train, y_test = load_data(10)

100%|██████████| 10/10 [00:00<00:00, 242.87it/s]


In [63]:
data.shape

(10, 210, 210, 3)

In [64]:
y.shape

(10, 1)

In [23]:
data_2 = load_data(10, random = True)

100%|██████████| 10/10 [00:00<00:00, 32.97it/s]


In [24]:
data_2.shape

(10, 210, 210, 3)

__Load LR data__

In [34]:
def load_data_lr(sample = None, size = 210):
    # > load y data 
    filepath = os.path.join("..", "in", "processed_data", "y_data.npy")
    # Load array
    y = np.load(filepath)
    
    # > Load file_list to be certain that X data will be in the same order as y 
    # Get the filepath
    filepath = os.path.join("..", "in", "processed_data", "file_list_img.csv")
    # Reading the filepath 
    file_list = pd.read_csv(filepath)
    
    # Choose to sample or not
    if sample: 
        # Convert y to dataframe to sample
        y_df = pd.DataFrame(y, columns =["label"])
        # Sample and get index
        y_sample = (y_df.groupby("label", as_index=False)
                    .apply(lambda x: x.sample(n=sample, replace=False).index)
                    .reset_index(drop=True))
        # Convert 2d list if indexes into 1d
        flatten_list = list(chain.from_iterable(y_sample))
        # Use indexes to find the y values in the sample
        y_relevant = np.array([y[i] for i in flatten_list])
        
        # Define list of files to iretate over
        y_filenames = [file_list["files"].tolist()[i] for i in flatten_list]
    else:
        # Define list of files to iretate over
        y_filenames = file_list["files"].tolist()
        # Define relevant y values
        y_relevant = y
    
    # Define empthy list 
    X = []
    # Iterate over images to load as arrays
    for file in tqdm(y_filenames[:100]):
        # Get filepath
        filepath = os.path.join("..", "in", "images", file)
        # Load image
        image = cv2.imread(filepath)
        # Convert to greyscale
        gray = cv2.bitwise_not(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        # Resize image
        compressed_gray = cv2.resize(gray, (int(size), int(size)), interpolation=cv2.INTER_AREA)
        # Append to list
        X.append(compressed_gray.flatten())

    # Making sure that both X and y are numpy arrays
    X = np.array(X)
    y_relevant = np.array(y_relevant)

    # Splitting data 
    X_train, X_test, y_train, y_test = train_test_split(X, y_relevant[:100],
                                                    random_state=42,
                                                    test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

In [35]:
X_train, X_test, y_train, y_test = load_data_lr()

100%|██████████| 100/100 [00:00<00:00, 710.72it/s]


In [224]:
X_train.shape

(448, 44100)

In [225]:
X_test.shape

(112, 44100)

In [227]:
y_train.shape

(448, 1)

In [230]:
y_test.shape

(112, 1)

In [231]:
y_test[:10]

array([[22],
       [17],
       [ 8],
       [ 4],
       [16],
       [ 7],
       [13],
       [14],
       [ 5],
       [25]])

__Normalize__

In [232]:
# > Normalize data
def normalize(X_train, X_test):
    # Scaling the features
    X_train_scaled = X_train / 255
    X_test_scaled = X_test / 255
    
    return (X_train_scaled, X_test_scaled)

In [233]:
X_train, X_test = normalize(X_train, X_test)

__Hold up I am not finised with loading data yet__

In [28]:
# > load y data 
filepath = os.path.join("..", "in", "processed_data", "y_data.npy")
# Load array
y = np.load(filepath)

In [188]:
# df.groupby('category')['item'].apply(lambda s: s.sample(min(len(s), 10)))
# 181 is the max
y_df = pd.DataFrame(y, columns =["label"])
y_sample = (y_df.groupby("label", as_index=False)
            .apply(lambda x: x.sample(n=400, replace=False).index)
            .reset_index(drop=True))

In [29]:
len(y_sample)

NameError: name 'y_sample' is not defined

In [30]:
# Lowest nr is 276
for i in range(28):
    print(np.count_nonzero(y == i))

1368
357
298
1196
1024
582
315
811
335
1074
759
9020
1073
783
1055
1128
3674
302
398
319
276
1138
1105
362
1899
319
569
714


In [190]:
# import chain
from itertools import chain
 
# converting 2d list into 1d
# using chain.from_iterables
flatten_list = list(chain.from_iterable(y_sample))

In [213]:
max(flatten_list)

32967

In [192]:
res_list = np.array([y[i] for i in flatten_list])

In [206]:
np.count_nonzero(res_list == 20)

400

In [214]:
l = file_list["files"].tolist()

In [215]:
len(y)

32968

In [209]:
len(l)

32253

In [210]:
y_filenames = [l[i] for i in flatten_list]

IndexError: list index out of range

__Training model__ 

In [234]:
# > Train model
def train_model(X_train, y_train):
    # Print info
    print("[info] Training model...")
    # Initialyzing model
    clf = LogisticRegression(multi_class="multinomial")
    # Training model 
    clf = LogisticRegression(penalty="none",
                             tol=0.1,
                             solver="saga",
                             multi_class="multinomial").fit(X_train, y_train.ravel()) 
                             # y_train.ravel() is to make it easier for the model to parse the y data of the cifar dataset
    return clf

In [235]:
# Training model 
clf = train_model(X_train, y_train)

[info] Training model...


In [87]:
y_train.unique

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [236]:
# > Report
def report(clf, X_test, y_test, label_names):
    # Print info 
    print("[info] Reporting results...")
    # Predict classification of test data
    y_pred = clf.predict(X_test)
    # Get metrics
    report = metrics.classification_report(y_test, 
                                           y_pred,
                                           target_names=label_names)
    # Print metrics
    print(report)
    # Save metrics
    #outpath = os.path.join("output", f"lr_report_{data}.txt")
    #with open(outpath, "w") as f:
        #f.write(report)


In [ ]:
# Reporting data 
    report(clf, X_test, y_test, label_names)
    